# Exploratory data analysis of a king county housing dataset

### My client

I have to explore the data and come up with insights and properties to sell for my client.
My client is a mafioso, so I can assume that she wants to keep a low profile and not do anything suspicious. She probably wants to sell renovated houses. It is given that she wants to sell top 10% houses in the central area. Furthermore, she wants to keep or buy houses which are remote and where she can hide from law enforcement. 

### My hypotheses

1) Top 10% most expensive homes are concentrated in central areas.

2) In typical transaction patterns over time, high-priced homes are sold irregularly and not in quick succession.

3) House prices are higher at a certain time of the year.



# Preparations

First, we load all the libraries needed for our EDA.

In [ ]:
#import warnings
import warnings
warnings.filterwarnings("ignore")

#import plotting libraries
import pandas as pd
from datetime import datetime, date, time, timedelta
import numpy as np
import plotly.express as px
import plotly.graph_objects as go



# Fetching the data

We connect to the database using psychopg2, fetch the data and save the data into a CSV file. If you revisit this notebook, you will not have to fetch the data again.
Make sure you have inserted username and password into the .env file.

In [2]:
import psycopg2 #the library used to connect to the PostgreSQL database
import os
from dotenv import load_dotenv

load_dotenv() #loading the .env file

DATABASE = os.getenv('DATABASE') #loading our database parameters
USER_DB = os.getenv('USER_DB')
PASSWORD = os.getenv('PASSWORD')
HOST = os.getenv('HOST')
PORT = os.getenv('PORT')

# Create connection object conn
conn = psycopg2.connect(
    database=DATABASE,
    user=USER_DB,
    password=PASSWORD,
    host=HOST,
    port=PORT
)

In [3]:
# creating a cursor and connecting to the database
cur = conn.cursor()
# creating a SQL command to fetch two tables and join them
query_string = 'SELECT d.*, s."date", s.price FROM eda.king_county_house_details AS d FULL JOIN eda.king_county_house_sales AS s ON d.id = s.house_id;'
# creating a pandas dataframe from the SQL query
df = pd.read_sql(query_string, conn)
# closing the connection
conn.close()
# saving the data as cvs file
df.to_csv('data/king-county-eda.csv', index=False)

# A first look at the data

In [4]:
# import the data from csv
df = pd.read_csv('data/king-county-eda.csv')
# take a first look at the beginning of the table ...
df.head(10)

,id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,date,price
0,7129300520,3.0,1.00,1180.0,5650.0,1.0,NaN,0.0,3,7,...,0.0,1955,0.0,98178,47.5112,-122.257,1340.0,5650.0,2014-10-13,221900.0
1,6414100192,3.0,2.25,2570.0,7242.0,2.0,0.0,0.0,3,7,...,400.0,1951,19910.0,98125,47.7210,-122.319,1690.0,7639.0,2014-12-09,538000.0
2,5631500400,2.0,1.00,770.0,10000.0,1.0,0.0,0.0,3,6,...,0.0,1933,NaN,98028,47.7379,-122.233,2720.0,8062.0,2015-02-25,180000.0
3,2487200875,4.0,3.00,1960.0,5000.0,1.0,0.0,0.0,5,7,...,910.0,1965,0.0,98136,47.5208,-122.393,1360.0,5000.0,2014-12-09,604000.0
4,1954400510,3.0,2.00,1680.0,8080.0,1.0,0.0,0.0,3,8,...,0.0,1987,0.0,98074,47.6168,-122.045,1800.0,7503.0,2015-02-18,510000.0
5,7237550310,4.0,4.50,5420.0,101930.0,1.0,0.0,0.0,3,11,...,1530.0,2001,0.0,98053,47.6561,-122.005,4760.0,101930.0,2014-05-12,1230000.0
6,1321400060,3.0,2.25,1715.0,6819.0,2.0,0.0,0.0,3,7,...,NaN,1995,0.0,98003,47.3097,-122.327,2238.0,6819.0,2014-06-27,257500.0
7,2008000270,3.0,1.50,1060.0,9711.0,1.0,0.0,NaN,3,7,...,0.0,1963,0.0,98198,47.4095,-122.315,1650.0,9711.0,2015-01-15,291850.0
8,2414600126,3.0,1.00,1780.0,7470.0,1.0,0.0,0.0,3,7,...,730.0,1960,0.0,98146,47.5123,-122.337,1780.0,8113.0,2015-04-15,229500.0
9,3793500160,3.0,2.50,1890.0,6560.0,2.0,0.0,0.0,3,7,...,0.0,2003,0.0,98038,47.3684,-122.031,2390.0,7570.0,2015-03-12,323000.0


In [5]:
# ... and the end of the table
df.tail()

,id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,date,price
21592,263000018,3.0,2.50,1530.0,1131.0,3.0,0.0,0.0,3,8,...,0.0,2009,0.0,98103,47.6993,-122.346,1530.0,1509.0,2014-05-21,360000.0
21593,6600060120,4.0,2.50,2310.0,5813.0,2.0,0.0,0.0,3,8,...,0.0,2014,0.0,98146,47.5107,-122.362,1830.0,7200.0,2015-02-23,400000.0
21594,1523300141,2.0,0.75,1020.0,1350.0,2.0,0.0,0.0,3,7,...,0.0,2009,0.0,98144,47.5944,-122.299,1020.0,2007.0,2014-06-23,402101.0
21595,291310100,3.0,2.50,1600.0,2388.0,2.0,NaN,0.0,3,8,...,0.0,2004,0.0,98027,47.5345,-122.069,1410.0,1287.0,2015-01-16,400000.0
21596,1523300157,2.0,0.75,1020.0,1076.0,2.0,0.0,0.0,3,7,...,0.0,2008,0.0,98144,47.5941,-122.299,1020.0,1357.0,2014-10-15,325000.0


In [6]:
# check the shape of the table and the information on dataypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   bedrooms       21597 non-null  float64
 2   bathrooms      21597 non-null  float64
 3   sqft_living    21597 non-null  float64
 4   sqft_lot       21597 non-null  float64
 5   floors         21597 non-null  float64
 6   waterfront     19206 non-null  float64
 7   view           21534 non-null  float64
 8   condition      21597 non-null  int64  
 9   grade          21597 non-null  int64  
 10  sqft_above     21597 non-null  float64
 11  sqft_basement  21145 non-null  float64
 12  yr_built       21597 non-null  int64  
 13  yr_renovated   17749 non-null  float64
 14  zipcode        21597 non-null  int64  
 15  lat            21597 non-null  float64
 16  long           21597 non-null  float64
 17  sqft_living15  21597 non-null  float64
 18  sqft_l

# Data Cleaning

We can can see that there are some null values. Let's check if there are more data missing.

In [7]:
# check the number of missing values in each column
df.isna().sum()

id                  0
bedrooms            0
bathrooms           0
sqft_living         0
sqft_lot            0
floors              0
waterfront       2391
view               63
condition           0
grade               0
sqft_above          0
sqft_basement     452
yr_built            0
yr_renovated     3848
zipcode             0
lat                 0
long                0
sqft_living15       0
sqft_lot15          0
date                0
price               0
dtype: int64

There are missing values in: waterfront, view, sqft_basement and yr_renovated. We are going to replace the NaNs with zeroes creating a new dataframe.

In [8]:
# fill missing values with 0
df_2 = df.fillna(0)
# Let's check if this worked
df_2.isna().sum()

id               0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
grade            0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
zipcode          0
lat              0
long             0
sqft_living15    0
sqft_lot15       0
date             0
price            0
dtype: int64

We take a look at the yr_renovated column where the value is not 0. 

In [9]:
df_2.query('yr_renovated > 0')['yr_renovated'].head(20)


1      19910.0
35     20020.0
95     19910.0
103    20100.0
125    19920.0
158    20130.0
209    19940.0
216    19910.0
230    19780.0
237    20050.0
274    20030.0
282    19940.0
324    19840.0
325    19840.0
330    20020.0
358    19540.0
379    20140.0
398    20110.0
435    20140.0
470    19830.0
Name: yr_renovated, dtype: float64

We can see that the year is a factor 10 too high. We are going to divide all values by ten.

In [10]:
df_2['yr_renovated'] = df_2['yr_renovated'].apply(lambda x: x/10)
# check the first 20 values to see if the transformation worked
df_2['yr_renovated'].head(20)

0        0.0
1     1991.0
2        0.0
3        0.0
4        0.0
5        0.0
6        0.0
7        0.0
8        0.0
9        0.0
10       0.0
11       0.0
12       0.0
13       0.0
14       0.0
15       0.0
16       0.0
17       0.0
18       0.0
19       0.0
Name: yr_renovated, dtype: float64

Also, some values are of type float, but they should be of type int, so let's fix this.

In [11]:
df_3 = df_2.astype({'yr_renovated': int, 'bedrooms': int, 'sqft_living': int, 'sqft_lot': int, 
                   'sqft_living': int, 'sqft_above': int, 'sqft_basement': int, 'sqft_living15': int, 'sqft_lot15': int, 
                   'view': int, 'waterfront': int, 'price': int})
# Let's see if that worked
df_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   bedrooms       21597 non-null  int64  
 2   bathrooms      21597 non-null  float64
 3   sqft_living    21597 non-null  int64  
 4   sqft_lot       21597 non-null  int64  
 5   floors         21597 non-null  float64
 6   waterfront     21597 non-null  int64  
 7   view           21597 non-null  int64  
 8   condition      21597 non-null  int64  
 9   grade          21597 non-null  int64  
 10  sqft_above     21597 non-null  int64  
 11  sqft_basement  21597 non-null  int64  
 12  yr_built       21597 non-null  int64  
 13  yr_renovated   21597 non-null  int64  
 14  zipcode        21597 non-null  int64  
 15  lat            21597 non-null  float64
 16  long           21597 non-null  float64
 17  sqft_living15  21597 non-null  int64  
 18  sqft_l

Another issue is the the format of the date column.

In [12]:
# Let's look at the type of individual values in the date column
print(type(df_3.date[0]))
print(df_3.date[0])
print(df_3.date[1])
print(df_3.date[5])

<class 'str'>
2014-10-13
2014-12-09
2014-05-12


The dates are of type string. Let's covert them to datetype format. 

In [13]:
df_3['date'] = pd.to_datetime(df_3['date'], format = '%Y-%m-%d')
# check if it worked
print(type(df_3.date[0]))

<class 'pandas._libs.tslibs.timestamps.Timestamp'>


Let's take another look at our dataframe.

In [14]:
df_3.head()

,id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,date,price
0,7129300520,3,1.00,1180,5650,1.0,0,0,3,7,...,0,1955,0,98178,47.5112,-122.257,1340,5650,2014-10-13,221900
1,6414100192,3,2.25,2570,7242,2.0,0,0,3,7,...,400,1951,1991,98125,47.7210,-122.319,1690,7639,2014-12-09,538000
2,5631500400,2,1.00,770,10000,1.0,0,0,3,6,...,0,1933,0,98028,47.7379,-122.233,2720,8062,2015-02-25,180000
3,2487200875,4,3.00,1960,5000,1.0,0,0,5,7,...,910,1965,0,98136,47.5208,-122.393,1360,5000,2014-12-09,604000
4,1954400510,3,2.00,1680,8080,1.0,0,0,3,8,...,0,1987,0,98074,47.6168,-122.045,1800,7503,2015-02-18,510000


Let's sort the data according to id, then sell date.

In [15]:
df_4 = df_3.sort_values(['id', 'date'])

In [16]:
df_4.head(10)

,id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,date,price
2494,1000102,6,3.00,2400,9373,2.0,0,0,3,7,...,0,1991,0,98002,47.3262,-122.214,2060,7316,2014-09-16,280000
2495,1000102,6,3.00,2400,9373,2.0,0,0,3,7,...,0,1991,0,98002,47.3262,-122.214,2060,7316,2015-04-22,300000
6729,1200019,4,1.75,2060,26036,1.0,0,0,4,8,...,900,1947,0,98166,47.4444,-122.351,2590,21891,2014-05-08,647500
8404,1200021,3,1.00,1460,43000,1.0,0,0,3,7,...,0,1952,0,98166,47.4434,-122.347,2250,20023,2014-08-11,400000
8800,2800031,3,1.00,1430,7599,1.5,0,0,4,6,...,420,1930,0,98168,47.4783,-122.265,1290,10320,2015-04-01,235000
3553,3600057,4,2.00,1650,3504,1.0,0,0,3,7,...,890,1951,2013,98144,47.5803,-122.294,1480,3504,2015-03-19,402500
18506,3600072,4,2.75,2220,5310,1.0,0,0,5,7,...,1050,1951,0,98144,47.5801,-122.294,1540,4200,2015-03-30,680000
3197,3800008,5,1.50,1990,18200,1.0,0,0,3,7,...,0,1960,0,98178,47.4938,-122.262,1860,8658,2015-02-24,178000
21047,5200087,4,2.50,2540,5001,2.0,0,0,3,9,...,0,2005,0,98108,47.5423,-122.302,2360,6834,2014-07-09,487000
4329,6200017,3,1.00,1340,21336,1.5,0,0,4,5,...,0,1945,0,98032,47.4023,-122.273,1340,37703,2014-11-12,281000


It seems our data is clean and sorted! Let's store our clean data in a separate CSV file. If you revisit the EDA part again you will not have to run the cleaning steps.

In [17]:
df_4.to_csv('data/king-county-eda_cleaned.csv', index=False) 

# Exploratory Data Analysis

Now we explore our clean data by EDA. We first load the data from our CSV file.

In [18]:
#import the clean data from a csv-file
df = pd.read_csv('data/king-county-eda_cleaned.csv')
#set the datetime format for the date column, as this information is lost. 
df['date'] = pd.to_datetime(df['date'], format = '%Y-%m-%d')

### Duplicate IDs

We want to see wether some houses are on the list more than once.

In [19]:
# Duplicated values
df['id'].duplicated().sum() 

np.int64(177)

We found that we have 177 duplicated entries! Let's take a brief look...

In [20]:
# we group by the id column to see if we have duplicates, triplicates, etc.
df_groupid = df.groupby('id').count().sort_values('price', ascending=False)
df_groupid.head()

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,date,price
id,,,,,,,,,,,,,,,,,,,,
795000620,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
1000102,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
3578401060,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
3528000040,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
4345000510,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2


One ID is there three times, and the others are duplicates. Let's briefly look at the triplicates. 

In [21]:
df.loc[df['id'] == 795000620] 

,id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,date,price
1618,795000620,3,1.0,1080,6250,1.0,0,0,2,5,...,0,1950,0,98168,47.5045,-122.33,1070,6250,2014-09-24,115000
1619,795000620,3,1.0,1080,6250,1.0,0,0,2,5,...,0,1950,0,98168,47.5045,-122.33,1070,6250,2014-12-15,124000
1620,795000620,3,1.0,1080,6250,1.0,0,0,2,5,...,0,1950,0,98168,47.5045,-122.33,1070,6250,2015-03-11,157000


This house was sold three times in a short period of time, three months each. The price increased from 1.2 million to 1.6 million. This looks suspicious, I wonder if this was my client. 

Let's drop all the duplicates from the table and only keep the latest instance.

In [22]:
# Drop the oldest duplicate ids
# Sort by 'date' descending so newest comes first
df2 = df.sort_values(by='date', ascending=False)

# Drop duplicates based on 'id', keeping the first (i.e., newest date)
df2 = df2.drop_duplicates(subset='id', keep='first').reset_index(drop=True)

# chekc if duplicates are dropped
df2['id'].duplicated().sum() # no duplicates

np.int64(0)

### Price distribution
As our client is interested in the top 10% houses, let's check the price distribution.

In [23]:
df2.price.describe()

count    2.142000e+04
mean     5.418614e+05
std      3.675569e+05
min      7.800000e+04
25%      3.249500e+05
50%      4.505500e+05
75%      6.450000e+05
max      7.700000e+06
Name: price, dtype: float64

In [89]:
px.histogram(df2, x='price', nbins=100, height=500, width=600, template='plotly_white', title='Price Distribution').update_layout(title_x=0.5)

### Identifying outliers
We can see the price distribution is skewed right and there might be some outliers. Let's look at the boxplot to identify outliers.

In [88]:
fig = px.box(df2, y="price", title="Price Boxplot")
fig.update_traces(marker_color='blue', marker_line_color='black')
fig.update_layout(yaxis_title="Price", xaxis_title="Price Boxplot", height=500, width=500, template='plotly_white', title='Price Distribution', title_x=0.5)
fig.show()

We would have to exclude too many values this way, so we will choose the outliers by checking the histogram.

In [87]:
px.histogram(df2, x='price', range_x = [2000000, 8000000], range_y= [0, 18], nbins=300, height=500, width=1000, template='plotly_white', title='Price Distribution').update_layout(title_x=0.5)

From this we will choose a cutoff of 3.5M USD. When we would sell those very expensive houses, we might raise suspicion.

Now let's drop these super expensive houses from our dataset.

In [27]:

df3 = df2.drop(df2[df2['price'] > 3500000].index)


### Top 10% houses
From this new list we want to narrow down the houses my client is interested in. These are the top 10% houses in the central area. 

Also we want to check if the top 10% houses are located in the central area anyway to test our hypothesis.

In [28]:
# we sort the values by price, grade and condition, then take the top 10% of the values
top10 = df3.sort_values(['price', 'grade', 'condition'], ascending=False).head(int(df3.price.count()*0.1)).reset_index(drop=True)
top10.price.describe()

count    2.139000e+03
mean     1.309225e+06
std      4.719572e+05
min      8.850000e+05
25%      9.700000e+05
50%      1.150000e+06
75%      1.470000e+06
max      3.420000e+06
Name: price, dtype: float64

So, the price range of the top 10 houses are: 885.000 - 3.420.000USD +/- 471.957USD with a mean value of 1309225USD. There are 2139 houses that fall into this category. Let us see if they are located in the central area. Let's make a map of all houses and a map of the 10% houses.

In [123]:
# Map of all houses 

# these are all the zip codes in King Co
zip_codes_list = [
    98052, 98105, 98092, 98115, 98023, 98103, 98003, 98042, 98034, 98133,
    98118, 98125, 98031, 98058, 98122, 98032, 98059, 98033, 98004, 98006,
    98001, 98117, 98030, 98056, 98155, 98198, 98168, 98002, 98038, 98109,
    98040, 98029, 98027, 98074, 98146, 98144, 98007, 98072, 98011, 98188,
    98008, 98107, 98106, 98178, 98116, 98075, 98108, 98112, 98028, 98119,
    98055, 98053, 98126, 98199, 98177, 98022, 98102, 98166, 98005, 98121,
    98136, 98045, 98101, 98104, 98077, 98057, 98065, 98019, 98070, 98148,
    98014, 98354, 98051, 98010, 98024, 98047, 98151, 98134, 98039,
    98158, 98195, 98288, 98174, 98054, 98132, 98171, 98184, 98224, 98164,
    98181, 98009, 98013, 98015, 98025, 98035, 98041, 98050, 98062, 98064,
    98063, 98071, 98073, 98083, 98089, 98093, 98111, 98113, 98114, 98124,
    98127, 98131, 98129, 98139, 98138, 98141, 98145, 98154, 98160, 98161,
    98165, 98170, 98175, 98185, 98190, 98194, 98191
]
#We sort the list of zip codes
#We make a dataframe from the list
zip_codes_list.sort()
df_all_zip_codes = pd.DataFrame(zip_codes_list, columns=['zipcode'])
df_all_zip_codes.head()


#This is a scatter map displaying all houses as a point on the map
fig1 = px.scatter_map(df2, #our dataframe
                      lat=df2['lat'], #set latitudes
                      lon=df2['long'], #set longitudes
                      hover_name = 'price', #set the name that appears when hovering with the cursor over the dots
                      hover_data={'lat':False, 'long':False, 'zipcode':True}, #sets additional hovering data (latitude and longitude is True by default)
                      center = {"lat": 47.5000, "lon": -122.000}, #this sets the center of King County as initial map center
                      zoom =8, #initial zoom level
                      color_discrete_sequence=['maroon'],
                      #color='price', #color of the dots'
                      ) #color of the dots

figa = px.choropleth_map(df_all_zip_codes, #the data (zipcode and price)
                         geojson='KingCo-Zip.geojson', #geojson file of King-County
                         color='zipcode', #the coloring of the states depends on the value in the iata column (number of airports)
                         color_continuous_scale="burgyl", #the color scheme
                         opacity=0.5, #set opacity
                         locations='zipcode', #the info of the locations used, in our case its the states 
                         featureidkey="properties.ZIP", #the key in the geojson file that corresponds to the locations
                         center = {"lat": 47.6000, "lon": -122.200},#this sets the center of the US as initial map center
                         height=600,
                         width=800,  
                         zoom=8,
                         )

    
fig_combined = go.Figure()

for trace in figa.data: 
    fig_combined.add_trace(trace),  #adding our choropleth map data

for trace in fig1.data: 
    fig_combined.add_trace(trace) #adding our scattermap data


fig_combined.update_layout(figa.layout, title= 'King County Houses', title_x=0.5) 

#using the layout of one of the maps is important to set zoom and center. We use the one of the choropleth map, 
#otherwise our choropleth color scheme would be set to the default scheme.
fig_combined.show()

In [124]:
#Map of Top 10% houses

# these are all the zip codes in King Co
zip_codes_list = [
    98052, 98105, 98092, 98115, 98023, 98103, 98003, 98042, 98034, 98133,
    98118, 98125, 98031, 98058, 98122, 98032, 98059, 98033, 98004, 98006,
    98001, 98117, 98030, 98056, 98155, 98198, 98168, 98002, 98038, 98109,
    98040, 98029, 98027, 98074, 98146, 98144, 98007, 98072, 98011, 98188,
    98008, 98107, 98106, 98178, 98116, 98075, 98108, 98112, 98028, 98119,
    98055, 98053, 98126, 98199, 98177, 98022, 98102, 98166, 98005, 98121,
    98136, 98045, 98101, 98104, 98077, 98057, 98065, 98019, 98070, 98148,
    98014, 98354, 98051, 98010, 98024, 98047, 98151, 98134, 98039,
    98158, 98195, 98288, 98174, 98054, 98132, 98171, 98184, 98224, 98164,
    98181, 98009, 98013, 98015, 98025, 98035, 98041, 98050, 98062, 98064,
    98063, 98071, 98073, 98083, 98089, 98093, 98111, 98113, 98114, 98124,
    98127, 98131, 98129, 98139, 98138, 98141, 98145, 98154, 98160, 98161,
    98165, 98170, 98175, 98185, 98190, 98194, 98191
]
#We sort the list of zip codes
#We make a dataframe from the list
zip_codes_list.sort()
df_all_zip_codes = pd.DataFrame(zip_codes_list, columns=['zipcode'])
df_all_zip_codes.head()


#This is a scatter map displaying all houses as a point on the map
fig1 = px.scatter_map(top10, #our dataframe
                      lat=top10['lat'], #set latitudes
                      lon=top10['long'], #set longitudes
                      hover_name = 'price', #set the name that appears when hovering with the cursor over the dots
                      hover_data={'lat':False, 'long':False, 'zipcode': True}, #sets additional hovering data (latitude and longitude is True by default)
                      center = {"lat": 47.5000, "lon": -122.000}, #this sets the center of King County as initial map center
                      zoom =8, #initial zoom level
                      color_discrete_sequence=['maroon'],
                      #color='price', #color of the dots'
                      ) #color of the dots
#As a background we have all the zip code areas
figa = px.choropleth_map(df_all_zip_codes, #the data (zipcode and price)
                         geojson='KingCo-Zip.geojson', #geojson file of King-County
                         color='zipcode', #the coloring of the states depends on the value in the iata column (number of airports)
                         color_continuous_scale="burgyl", #the color scheme
                         opacity=0.5, #set opacity
                         locations='zipcode', #the info of the locations used, in our case its the states 
                         featureidkey="properties.ZIP", #the key in the geojson file that corresponds to the locations
                         center = {"lat": 47.6000, "lon": -122.200},#this sets the center of the US as initial map center
                         height=600,
                         width=800,  
                         zoom=8 )

    
fig_combined = go.Figure()

for trace in figa.data: 
    fig_combined.add_trace(trace),  #adding our choropleth map data

for trace in fig1.data: 
    fig_combined.add_trace(trace) #adding our scattermap data


fig_combined.update_layout(figa.layout, title= 'Top 10% King County Houses', title_x=0.5) 

#using the layout of one of the maps is important to set zoom and center. We use the one of the choropleth map, 
#otherwise our choropleth color scheme would be set to the default scheme.
fig_combined.show()

We can see that most of the top 10% houses are located around Seattle city centre. 
To narrow down our search, we want to see how many of these houses are renovated, and hwo many are not.

In [31]:
print('Of the top 10% houses,', top10.query('yr_renovated > 0')['yr_renovated'].count(), 'houses are renovated and',
      top10.query('yr_renovated == 0')['yr_renovated'].count(), 'are not renovated.')

Of the top 10% houses, 189 houses are renovated and 1950 are not renovated.


Let's make a pie chart:

In [128]:
#We create a new data frame
df_renovation = pd.DataFrame()
#and make a column which is separated into two categories: renovated and not renovated, according to our top 10 dataframe
df_renovation['renov_status'] = top10['yr_renovated'].apply(lambda x: 'renovated' if x > 0 else 'not renovated')

px.pie(df_renovation, names='renov_status', 
       title='Renovation status of top 10% houses',
       color_discrete_sequence=px.colors.qualitative.Safe, height=400, width=400).update_layout(title_x=0.5,paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)'
)

So we have narrowed down our search to 189 houses, let's make a new dataframe for the top10% renovated houses and inspect.

In [33]:
#remove the unrenovated houses
top10renov = top10.drop(top10[top10['yr_renovated'] == 0].index)

top10renov.head(10)

,id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,date,price
1,1118000320,4,4.00,4260,11765,2.0,0,0,3,11,...,980,1939,2010,98112,47.6380,-122.288,4260,10408,2015-05-08,3400000
6,3625059152,3,3.25,4220,41300,1.0,1,4,4,11,...,1760,1958,1987,98008,47.6083,-122.110,3810,30401,2014-12-30,3300000
10,1068000375,6,5.00,7100,18200,2.5,0,0,3,13,...,1860,1933,2002,98199,47.6427,-122.408,3130,6477,2014-09-23,3200000
28,9808100100,5,3.25,5370,14091,2.0,0,0,3,10,...,1520,1918,2008,98004,47.6499,-122.216,2410,12047,2015-02-02,3000000
48,7397300220,4,3.25,4430,21000,2.0,0,0,3,10,...,0,1952,2007,98039,47.6398,-122.237,3930,20000,2014-05-29,2750000
54,5442300807,5,2.75,3831,13800,2.0,1,4,3,9,...,0,1959,1980,98040,47.5919,-122.251,3850,36563,2014-06-24,2700000
57,3126059027,4,3.50,4700,13730,2.0,0,3,3,11,...,1200,1958,1995,98033,47.6899,-122.217,3210,15306,2015-03-18,2650000
72,3377900195,4,5.50,6930,45100,1.0,0,0,4,11,...,0,1950,1991,98006,47.5547,-122.144,2560,37766,2014-09-29,2530000
77,3025300095,4,4.50,4300,19844,2.0,0,0,3,11,...,0,1985,1999,98039,47.6218,-122.237,3070,19845,2014-10-09,2500000
79,4107100190,4,3.75,3480,14850,1.0,0,4,3,9,...,1610,1951,2013,98004,47.6227,-122.216,4780,18480,2015-03-24,2500000


Let's see how many houses have waterfront, and how the condition is of the houses. Waterfront houses will be very popular, easy to sell for a high price. 

In [91]:
px.bar(top10renov.groupby('waterfront').count().reset_index(), x='waterfront', y='id', height=400, width=400, title= 'Waterfront Houses').update_layout(title_x=0.5)

We will take a look at those waterfront houses by looking at hwo many houses are in which condition category, since we want to have houses with a good condition.

In [92]:
top10renovwaterfront = top10renov.query('waterfront == 1')
# we group by category and see how may houses are in which category
px.bar(top10renovwaterfront.groupby('condition').count().reset_index(), x='condition', y='id', height=400, width=400, title=('Condition of Waterfront Houses')).update_layout(title_x=0.5)

We see that 5 houses are in condition 4 and one in condition 5. We will drop the houses with condition 3. 

In [126]:
#we drop all houses where the condition is less than 4
top10renovwaterfront = top10renovwaterfront.drop(top10renovwaterfront[top10renovwaterfront['condition'] < 4].index).reset_index(drop=True)
top10renovwaterfront

,id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,date,price,month
0,3625059152,3,3.25,4220,41300,1.0,1,4,4,11,...,1958,1987,98008,47.6083,-122.110,3810,30401,2014-12-30,3300000,12
1,7352200450,4,3.25,3580,19989,1.5,1,4,4,7,...,1915,1965,98125,47.7087,-122.276,2410,6389,2015-01-15,2050000,1
2,3343302110,3,3.00,2790,13295,2.0,1,4,4,10,...,1933,1989,98006,47.5466,-122.197,3140,11949,2015-03-06,1800000,3
3,7204200025,4,2.50,3120,49456,2.0,1,4,4,9,...,1974,1989,98198,47.3535,-122.323,2030,32181,2014-10-28,1230000,10
4,7352200025,2,1.75,2080,8112,1.0,1,4,4,8,...,1939,1984,98125,47.7134,-122.277,2030,8408,2014-10-13,1190000,10
5,3343301910,5,4.50,2120,8944,2.0,1,4,5,8,...,1939,1963,98006,47.5488,-122.197,2870,8944,2014-10-20,1000000,10


We are left with 6 houses now, we cen see them on a map:

In [129]:
#This is a scatter map displaying the top 10 waterfront with condition above 3
fig1 = px.scatter_map(top10renovwaterfront, #our dataframe
                      lat=top10renovwaterfront['lat'], #set latitudes
                      lon=top10renovwaterfront['long'], #set longitudes
                      hover_name = 'price', #set the name that appears when hovering with the cursor over the dots
                      hover_data={'lat':False, 'long':False, 'zipcode': True}, #sets additional hovering data (latitude and longitude is True by default)
                      center = {"lat": 47.5000, "lon": -122.000}, #this sets the center of King County as initial map center
                      zoom =8, #initial zoom level
                      color_discrete_sequence=['red'],
                      #color='price', #color of the dots'
                      ) #color of the dots
#As a background we have all the zip code areas
figa = px.choropleth_map(df_all_zip_codes, #the data (zipcode and price)
                         geojson='KingCo-Zip.geojson', #geojson file of King-County
                         color='zipcode', #the coloring of the states depends on the value in the iata column (number of airports)
                         color_continuous_scale="burgyl", #the color scheme
                         opacity=0.5, #set opacity
                         locations='zipcode', #the info of the locations used, in our case its the states 
                         featureidkey="properties.ZIP", #the key in the geojson file that corresponds to the locations
                         center = {"lat": 47.6000, "lon": -122.200},#this sets the center of the US as initial map center
                         height=600,
                         width=800,  
                         zoom=8 )

    
fig_combined = go.Figure()

for trace in figa.data: 
    fig_combined.add_trace(trace),  #adding our choropleth map data

for trace in fig1.data: 
    fig_combined.add_trace(trace) #adding our scattermap data


fig_combined.update_layout(figa.layout, title='Houses for Client to Sell', title_x=0.5) 

#using the layout of one of the maps is important to set zoom and center. We use the one of the choropleth map, 
#otherwise our choropleth color scheme would be set to the default scheme.
fig_combined.show()

In [133]:
top10renovwaterfront

,id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,date,price,month
0,3625059152,3,3.25,4220,41300,1.0,1,4,4,11,...,1958,1987,98008,47.6083,-122.110,3810,30401,2014-12-30,3300000,12
1,7352200450,4,3.25,3580,19989,1.5,1,4,4,7,...,1915,1965,98125,47.7087,-122.276,2410,6389,2015-01-15,2050000,1
2,3343302110,3,3.00,2790,13295,2.0,1,4,4,10,...,1933,1989,98006,47.5466,-122.197,3140,11949,2015-03-06,1800000,3
3,7204200025,4,2.50,3120,49456,2.0,1,4,4,9,...,1974,1989,98198,47.3535,-122.323,2030,32181,2014-10-28,1230000,10
4,7352200025,2,1.75,2080,8112,1.0,1,4,4,8,...,1939,1984,98125,47.7134,-122.277,2030,8408,2014-10-13,1190000,10
5,3343301910,5,4.50,2120,8944,2.0,1,4,5,8,...,1939,1963,98006,47.5488,-122.197,2870,8944,2014-10-20,1000000,10


Let's see when would be the best time to sell. We look at the average house price per month of the year.

In [38]:
#Let's make a pivot table of date (month) and price of all houses of the dataset without the outliers

df3['month'] = df3['date'].dt.month #create a new column with the month of the date

date_price = pd.pivot_table(df3, values='price', index='month', aggfunc=['mean', 'std']).reset_index() #date vs. mean price pivot table

date_price.columns = ['month', 'mean', 'std']
date_price

,month,mean,std
0,1,524360.144033,353534.883472
1,2,508947.491158,326411.699534
2,3,544057.683200,355261.128306
3,4,556549.238994,335430.936503
4,5,547583.643096,353912.850736
5,6,552063.091248,340944.189303
6,7,545520.745197,340182.349828
7,8,535118.062271,342159.419706
8,9,526413.546445,324008.717223
9,10,532098.276757,330509.035879


Let's create a lineplot from this. 

In [134]:
fig = go.Figure()
# Here we could show the standard deviation as well, but it is very high and not very informative
#fig.add_trace(go.Scatter(
    #x=date_price['month'],
    #y=date_price['mean'] + date_price['std'],
    #mode='lines',
    #line=dict(width=0),
    #showlegend=False,
    #hoverinfo='skip',
    #name='Upper Bound'
#))
# fig.add_trace(go.Scatter(
#     x=date_price['month'],
#     y=date_price['mean'] - date_price['std'],
#     mode='lines',
#     fill='tonexty',  # fill between this and the previous trace
#     fillcolor='rgba(0,100,80,0.2)',
#     line=dict(width=0),
#     showlegend=True,
#     name='±1 Std Dev',
#     hoverinfo='skip'
# ))
# Add mean line
fig.add_trace(go.Scatter(
    x=date_price['month'],
    y=date_price['mean'],
    mode='lines+markers',
    name='Mean',
    line=dict(color='green'),
    marker=dict(size=8)
))
# Update layout
fig.update_layout(
    title='Average Price per Month all Houses',
    yaxis_title='Price (USD)',
    xaxis_title='Month',
    yaxis=dict(range=[480000, 580000]),  # Adjust y-axis range
    template='plotly_white',
    height=500,
    width=700,
    title_x=0.5,
)


Let's do the same for the top10 houses and the top10 renovated houses

In [40]:
#Let's make a pivot table of date (month) and price of the top 10% houses of the dataset without the outliers

top10['month'] = top10['date'].dt.month #create a new column with the month of the date

date_price10 = pd.pivot_table(top10, values='price', index='month', aggfunc=['mean', 'std']).reset_index() #date vs. mean price pivot table
date_price10.columns = ['month', 'mean', 'std']

In [136]:
fig = go.Figure()
# Here we could show the standard deviation as well, but it is very high and not very informative
# fig.add_trace(go.Scatter(
#     x=date_price10['month'],
#     y=date_price10['mean'] + date_price10['std'],
#     mode='lines',
#     line=dict(width=0),
#     showlegend=False,
#     hoverinfo='skip',
#     name='Upper Bound'
# ))
# fig.add_trace(go.Scatter(
#     x=date_price10['month'],
#     y=date_price10['mean'] - date_price10['std'],
#     mode='lines',
#     fill='tonexty',  # fill between this and the previous trace
#     fillcolor='rgba(0,100,80,0.2)',
#     line=dict(width=0),
#     showlegend=True,
#     name='±1 Std Dev',
#     hoverinfo='skip'
# ))
# Add mean line
fig.add_trace(go.Scatter(
    x=date_price10['month'],
    y=date_price10['mean'],
    mode='lines+markers',
    name='Mean',
    line=dict(color='orange'),
    marker=dict(size=8)
))
# Update layout
fig.update_layout(
    title='Average Price of Top 10% Houses per Month',
    yaxis_title='Price (USD)',
    xaxis_title='Month',
    yaxis=dict(range=[1100000, 1500000]),  # Adjust y-axis range
    template='plotly_white',
    height=500,
    width=700,
    title_x=0.5,
)

So, client should sell her houses in January, February and December as the top 10% houses are most expensive then, August is also a good month.

### Top10% vs. Bottom 90% sales
Let's see when the top10% houses are sold in comparison to the bottom 90% houses. Let's first quickly make a bottom 90 dataframe.

In [45]:

bottom90 = df3.sort_values(['price', 'grade', 'condition'], ascending=False).tail(int(df3.price.count()*0.9)).reset_index(drop=True)



Let's see how many houses were sold each day over time for both categories.

In [46]:
# we group top10 and bottom 90 by sale date
top10sales = top10.groupby('date').count().reset_index()
bottom90sales = bottom90.groupby('date').count().reset_index()

In [47]:
bottom90sales.id.describe()

count    370.000000
mean      52.048649
std       34.881492
min        1.000000
25%        7.250000
50%       63.500000
75%       79.000000
max      132.000000
Name: id, dtype: float64

In [48]:
top10sales.id.describe()

count    303.000000
mean       7.059406
std        4.260915
min        1.000000
25%        4.000000
50%        7.000000
75%        9.500000
max       21.000000
Name: id, dtype: float64

In [49]:
fig = go.Figure()

fig.add_trace(go.Histogram(x=top10sales['id'], name = 'Top 10%', nbinsx = 10, marker_color='red'))
fig.add_trace(go.Histogram(x=bottom90sales['id'], name = 'Bottom 90%', nbinsx = 30, marker_color='lightblue'))
# Overlay both histograms
fig.update_layout(barmode='overlay', 
                  title='Number of houses sold per day',
                  xaxis_title='Sales',
                  yaxis_title='Count',
                  height=500,
                  width=600,
                  template='plotly_white')
# Reduce opacity to see both histograms
fig.show()

In [143]:
fig = go.Figure()
fig.add_bar(x=top10sales['date'], y=top10sales['id'], name='Top 10% houses', marker_color='red')
fig.add_bar(x=bottom90sales['date'], y=bottom90sales['id'], name='Bottom 90% houses', marker_color='lightblue')
fig.update_layout(barmode='stack', 
                  title='House Sales per Day',
                  title_x=0.5,
                  xaxis_title='Date',
                  yaxis_title='Number of Houses Sold',
                  template='plotly_white',
                  height=500,
                  width=1800,)  

In comparison, there are far fewer houses sold which are in the top 10 range, and in whole King county maximally 21 houses a day with a mean of 7, while the maximum for bottom 90 houses is 132 houses sold maximally with a mean of 52 houses per day. So I confirm the hypothesis that top10% houses are sold less per day, but looking at the distribution throughout the week, they are sold regularly throughout the year. The frequency is lower in the months December - February, which is the time when they are most expensive. 

# Average Outskirt House

Client need average outskirt house to hide. Let us define the average outskirt house according to location and price. Looking on the map, I define the following Zipcodes as outskirts:
980...
19, 14, 24, 65, 51, 22, 45, 224, 298
Let us show them on a map:


In [51]:
# these are all the zip codes in King Co
outskirts_zip_codes_list = [
    98019, 98014, 98024, 98065, 98051, 98022, 98045, 98224, 98298
]
#We sort the list of zip codes
#We make a dataframe from the list
outskirts_zip_codes_list.sort()
df_outskirts_zip_codes = pd.DataFrame(outskirts_zip_codes_list, columns=['zipcode'])
df_outskirts_zip_codes

,zipcode
0,98014
1,98019
2,98022
3,98024
4,98045
5,98051
6,98065
7,98224
8,98298


In [52]:
#we drop all houses which are not outskirt houses and make a new dataframe:
outskirtshouses = df3[df3['zipcode'].isin(outskirts_zip_codes_list)].reset_index(drop=True)
outskirtshouses.query( 'zipcode == 98051').count()

id               0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
grade            0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
zipcode          0
lat              0
long             0
sqft_living15    0
sqft_lot15       0
date             0
price            0
month            0
dtype: int64

In [144]:

#This is a scatter map displaying all outskirts houses as a point on the map
fig1 = px.scatter_map(outskirtshouses, #our dataframe
                      lat=outskirtshouses['lat'], #set latitudes
                      lon=outskirtshouses['long'], #set longitudes
                      hover_name = 'price', #set the name that appears when hovering with the cursor over the dots
                      hover_data={'lat':False, 'long':False, 'zipcode': True}, #sets additional hovering data (latitude and longitude is True by default)
                      center = {"lat": 47.5000, "lon": -122.000}, #this sets the center of King County as initial map center
                      zoom =8, #initial zoom level
                      color_discrete_sequence=['maroon'],
                      #color='price', #color of the dots'
                      ) #color of the dots

figa = px.choropleth_map(df_all_zip_codes, #the data (zipcode and price)
                         geojson='KingCo-Zip.geojson', #geojson file of King-County
                         color='zipcode', #the coloring of the states depends on the value in the iata column (number of airports)
                         color_continuous_scale="burgyl", #the color scheme
                         opacity=0.5, #set opacity
                         locations='zipcode', #the info of the locations used, in our case its the states 
                         featureidkey="properties.ZIP", #the key in the geojson file that corresponds to the locations
                         center = {"lat": 47.6000, "lon": -122.200},#this sets the center of the US as initial map center
                         height=600,
                         width=800,  
                         zoom=8 )


    
fig_combined = go.Figure()

for trace in figa.data: 
    fig_combined.add_trace(trace),  #adding our choropleth map data

for trace in fig1.data: 
    fig_combined.add_trace(trace) #adding our scattermap data


fig_combined.update_layout(figa.layout, title= 'Outskirts Houses', title_x=0.5) 

#using the layout of one of the maps is important to set zoom and center. We use the one of the choropleth map, 
#otherwise our choropleth color scheme would be set to the default scheme.
fig_combined.show()

We do not have any houses now in 98051, and looking back, the houses here were assigned the wrong zipcode in the list! We will leave them out of the analysis. Let us look at the price range of the outskirt houses.

In [54]:
outskirtshouses.price.describe()

count    1.148000e+03
mean     4.486640e+05
std      2.162458e+05
min      8.000000e+04
25%      3.088500e+05
50%      4.183500e+05
75%      5.337725e+05
max      2.000000e+06
Name: price, dtype: float64

In [55]:
# we determine the pricerange as mean ± std
print(outskirtshouses.price.mean() - outskirtshouses.price.std())
print(outskirtshouses.price.mean() + outskirtshouses.price.std())

232418.21499715003
664909.7588704458


The price of these houses has a mean of roughly 450.000USD. Let's look at the size of the house (sqft_living): 232418USD to 664909 USD.

In [56]:
outskirtshouses.sqft_living.describe()

count    1148.000000
mean     2201.993031
std       955.024609
min       430.000000
25%      1540.000000
50%      2040.000000
75%      2710.000000
max      7420.000000
Name: sqft_living, dtype: float64

In [57]:
# we determine the sizerange as mean ± std
print(outskirtshouses.price.mean() - outskirtshouses.price.std())
print(outskirtshouses.price.mean() + outskirtshouses.price.std())

232418.21499715003
664909.7588704458


The size of the houses has a mean of roughly 2200 sqft. We want our sqft_living to be between 232418sqft and 664909sqft.

Let's narrow down our dataframe with these values, first the price, then the size.

In [58]:
# we drop the houses with a price above the mean + std and below the mean - std
avg_outstkirthouses = outskirtshouses.drop(outskirtshouses[outskirtshouses['price'] > outskirtshouses.price.mean() + outskirtshouses.price.std()].index).reset_index(drop=True)
avg_outstkirthouses = avg_outstkirthouses.drop(avg_outstkirthouses[avg_outstkirthouses['price'] < avg_outstkirthouses.price.mean() - avg_outstkirthouses.price.std()].index).reset_index(drop=True)

# we drop the houses with a sqft_living above the mean + std and below the mean - std
avg_outstkirthouses = avg_outstkirthouses.drop(avg_outstkirthouses[avg_outstkirthouses['sqft_living'] > avg_outstkirthouses.sqft_living.mean() + avg_outstkirthouses.sqft_living.std()].index).reset_index(drop=True)
avg_outstkirthouses = avg_outstkirthouses.drop(avg_outstkirthouses[avg_outstkirthouses['sqft_living'] < avg_outstkirthouses.sqft_living.mean() - avg_outstkirthouses.sqft_living.std()].index).reset_index(drop=True)

avg_outstkirthouses.id.count()


np.int64(584)

We are left with 584 potential houses. We will further narrow down the dataframe to only have the houses with basements, since basements are probably really handy for hiding things and digging secret escape tunnels.

In [59]:
avg_outstkirthouses_basement = avg_outstkirthouses.drop(avg_outstkirthouses[avg_outstkirthouses['sqft_basement'] == 0].index).reset_index(drop=True)
avg_outstkirthouses_basement.id.count()

np.int64(80)

We are left with 80 houses. Let's further narrow this down by removing all houses with a condition below 4.

In [60]:
Deluxehiding = avg_outstkirthouses_basement.drop(avg_outstkirthouses_basement[avg_outstkirthouses_basement['condition'] < 4].index).reset_index(drop=True)
Deluxehiding.id.count()

np.int64(20)

We are left with 20 houses. Let's look on the map.

In [149]:
#This is a scatter map displaying all outskirts houses as a point on the map
fig1 = px.scatter_map(Deluxehiding, #our dataframe
                      lat=Deluxehiding['lat'], #set latitudes
                      lon=Deluxehiding['long'], #set longitudes
                      hover_name = 'id', #set the name that appears when hovering with the cursor over the dots
                      hover_data={'lat':True, 'long':True, 'zipcode': True}, #sets additional hovering data (latitude and longitude is True by default)
                      center = {"lat": 47.5000, "lon": -122.000}, #this sets the center of King County as initial map center
                      zoom =8, #initial zoom level
                      color_discrete_sequence=['maroon'],
                      #color='price', #color of the dots'
                      ) #color of the dots

fig1.add_trace(go.Scattermap(
    lon=[-121.898, -121.93, -121.321],  
    lat=[47.7698, 47.7624, 47.7139],    
    mode='markers',
    line=dict(color='red', width=4),
    showlegend=False))

figa = px.choropleth_map(df_all_zip_codes, #the data (zipcode and price)
                         geojson='KingCo-Zip.geojson', #geojson file of King-County
                         color='zipcode', #the coloring of the states depends on the value in the iata column (number of airports)
                         color_continuous_scale="burgyl", #the color scheme
                         opacity=0.5, #set opacity
                         locations='zipcode', #the info of the locations used, in our case its the states 
                         featureidkey="properties.ZIP", #the key in the geojson file that corresponds to the locations
                         center = {"lat": 47.6000, "lon": -122.200},#this sets the center of the US as initial map center
                         height=600,
                         width=800,  
                         zoom=8 )


    
fig_combined = go.Figure()

for trace in figa.data: 
    fig_combined.add_trace(trace),  #adding our choropleth map data

for trace in fig1.data: 
    fig_combined.add_trace(trace) #adding our scattermap data


fig_combined.update_layout(figa.layout, title= 'Selection of Outskirt Houses for Client', title_x=0.5) 

#using the layout of one of the maps is important to set zoom and center. We use the one of the choropleth map, 
#otherwise our choropleth color scheme would be set to the default scheme.
fig_combined.show()

There are several options to choose from. When we look closely, we can see that three of the houses are in more remote areas in the mountains: 8649400410, 4047200380 and 1558100398.